# Apartado 1 Naive-Bayes propio
• Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (wdbc y heart). Considerar los dos tipos de particionado. Los
resultados se refieren a las tasas de error y deben mostrarse tanto con
la corrección de Laplace como sin ella. Se debe incluir tanto el
promedio de error como su desviación típica. Es importante mostrar
todos los resultados agrupados en una tabla para facilitar su evaluación.

In [1]:
import pandas as pd
import numpy as np
import random
import EstrategiaParticionado
from Datos import Datos
from Clasificador import ClasificadorNaiveBayes
from os import listdir

resultados = []
# Damos un valor aleatorio a la semilla para cada ejecución
seed = random.random()

# Ejecutamos cada dataset que se encuentre en la carpeta Datasets
for dataset_name in listdir('Datasets/'):
    dataset = Datos('Datasets/' + dataset_name)
    
    # Parámetros de las estrategias de particionado
    n_ejecuciones = 5
    n_folds = 5
    estrategia_simple = EstrategiaParticionado.ValidacionSimple(n_ejecuciones, 0.95)
    estrategia_cruzada = EstrategiaParticionado.ValidacionCruzada(n_folds)
    clasificador = ClasificadorNaiveBayes(laplace=1)

    # Con corrección de Laplace
    error_simple_laplace = clasificador.validacion(estrategia_simple, dataset, clasificador, seed)
    error_cruzada_laplace = clasificador.validacion(estrategia_cruzada, dataset, clasificador)
    
    # Sin corrección de Laplace
    clasificador.laplace = 0
    error_simple = clasificador.validacion(estrategia_simple, dataset, clasificador, seed)
    error_cruzada = clasificador.validacion(estrategia_cruzada, dataset, clasificador)
    
    # Calculamos los promedios y las desviaciones típicas y las almacenamos en los resutlados 
    # para posteriormente mostrarlos en una tabla
    resultados.append({
        'Dataset': dataset_name,
        'Particionado': 'Simple',
        'Laplace': True,
        'Error Promedio': np.mean(error_simple_laplace),
        'Desviación Típica': np.std(error_simple_laplace)
    })
    resultados.append({
        'Dataset': dataset_name,
        'Particionado': 'Simple',
        'Laplace': False,
        'Error Promedio': np.mean(error_simple),
        'Desviación Típica': np.std(error_simple)
    })
    resultados.append({
        'Dataset': dataset_name,
        'Particionado': 'Cruzada',
        'Laplace': True,
        'Error Promedio': np.mean(error_cruzada_laplace),
        'Desviación Típica': np.std(error_cruzada_laplace)
    })
    resultados.append({
        'Dataset': dataset_name,
        'Particionado': 'Cruzada',
        'Laplace': False,
        'Error Promedio': np.mean(error_cruzada),
        'Desviación Típica': np.std(error_cruzada)
    })

# Convertimos los resultados en un dataframe
df_resultados = pd.DataFrame(resultados)

# Mostramos la tabla
display(df_resultados)


,Dataset,Particionado,Laplace,Error Promedio,Desviación Típica
0,wdbc.csv,Simple,True,0.081701,0.015918
1,wdbc.csv,Simple,False,0.081701,0.015918
2,wdbc.csv,Cruzada,True,0.070315,0.031910
3,wdbc.csv,Cruzada,False,0.070315,0.031910
4,heart.csv,Simple,True,0.231881,0.013323
5,heart.csv,Simple,False,0.189220,0.017056
6,heart.csv,Cruzada,True,0.165716,0.054903
7,heart.csv,Cruzada,False,0.146110,0.053510


• Breve análisis de los resultados anteriores. Discutir el efecto Laplace.
Para la creación de particiones se han empleado parámetros estándar que son los que se encuentran por defecto en las versiones correspondientes de scikit learn, en concreto,
para la validación simple se ha utilizado un porcentaje del 75% (1 - 0.25) para el tamaño de la partición de entrenamiento y para la validación cruzada se han utilizado 5 folds.
Atendiendo a los resultados del conjunto de datos "wdbc.csv" podemos ver que tanto el error promedio como la desviación típica son muy bajos, indicando que la clasificación es casi perfecta teniendo una tasa de error promedio cercana al 5%, esto puede deberse a que los datos continuos siguen realmente una distribución Gaussiana y por lo tanto al suponer nuestro Naive Bayes una distribución Gaussiana para estos datos se obtiene este error tan bajo. Por otro lado, observamos que la corrección Laplaciana no afecta al resultado, esto se explica por el hecho de que el conjunto de datos unicamente cuenta con atributos continuos, y no aplicamos dicha corrección para atributos continuos. 
Analizando los resultados del conjunto "heart.csv" observamos unos ratios de error ligeramente más elevados que para el otro conjunto, esto posiblemente se pueda explicar por el hecho de que alguno de los atributos no siga una distribución Gaussiana, sin embargo, el error sigue siendo relativamente bajo rondando un 15%. En cuanto a Laplace para conjuntos de entrenamiento grandes no se observa ninguna diferencia significativa puesto que no hay datos que falten en el entrenamiento y Laplace no tiene efecto, sin embargo, si reducimos significativamente el tamaño del conjunto de entrenamiento podemos ver una diferencia alrededor del 2% entre aplicar el suavizado o no, siendo el NB con suavizado más preciso.